# Analyzing a Tennis Serve with Machine Learning

In this notebook you use the Video Intelligence API to analyze a tennis serve, including the angle of the arms and legs during the serve, and a pretrained custom model to track the speed of a tennis ball after being served.

## Setup

First, install the necessary libraries. The first cell will download and install the libraries onto the notebook, and the next cell imports them.

In [ ]:
!pip install google-cloud-automl

You might have to restart your runtime to load these packages.

In [ ]:
import sys
import os
import json
import math

import pandas as pd
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import numpy as np

from google.cloud import automl
from google.cloud import videointelligence_v1p3beta1 as videointelligence
from google.oauth2 import service_account

Fill in your info below by specifying your project id and bucket name. Replace YOUR_PROJECT_ID and YOUR_BUCKET_NAME with your Project ID as it's always a unique name.

In [ ]:
project_id = 'YOUR_PROJECT_ID'  #@param {type: "string"}
bucket = 'gs://YOUR_BUCKET_NAME-bucket' #@param {type: "string"}

Next, create your bucket, enable the video intelligence and AutoML APIs, and generate a service account key to access the AutoML API. For this lab, you will use your autogenerated Qwiklabs Service Account.

In [ ]:
!gcloud config set project {project_id}
!gsutil mb {bucket}
!gcloud iam service-accounts keys create ./key.json --iam-account {project_id}@{project_id}.iam.gserviceaccount.com

# Enable the Video Intelligence API and AutoML
!gcloud services enable videointelligence.googleapis.com
!gcloud services enable automl.googleapis.com

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./key.json"

## Using the Video Intelligence API

In this section you will analyze a skeleton using the Video Intelligence API. Using Person Detection, the Video Intelligence API can detect the presence of humans in a video file and track the bounding box of individual people across the video or video segment. Person Detection recognizes a handful of body parts, facial features, and clothing as "landmarks". A table from the documentation:

<div>
<img src="https://daleonai.com/images/screen-shot-2020-07-14-at-3-45-56-pm.png" width="800"/>
</div>

The API tracks all sorts of features, including facial features. For this lab you will only focus on the body.

![alt text](https://github.com/google/making_with_ml/blob/master/sports_ai/assets/posture_tracking.gif?raw=true))

To do this, you will use a prerecorded video of multiple tennis serves.

In [ ]:
input_uri = "gs://spls/aiforsports/tennis_serves.mp4"
output_uri = os.path.join(bucket, 'output.json')

Now, run the Video Intelligence API's Person Detection feature on the video from the bucket. This function takes an input path to your video and an output path where you want your results stored. 

In [ ]:
# This function comes from the docs
# https://cloud.google.com/video-intelligence/docs/people-detection
def detect_person(input_uri, output_uri):
    """Detects people in a video."""

    client = videointelligence.VideoIntelligenceServiceClient(credentials=service_account.Credentials.from_service_account_file(
    './key.json'))

    # Configure the request
    config = videointelligence.types.PersonDetectionConfig(
        include_bounding_boxes=True,
        include_attributes=True,
        include_pose_landmarks=True,
    )
    context = videointelligence.types.VideoContext(person_detection_config=config)

    # Start the asynchronous request
    operation = client.annotate_video(
        input_uri=input_uri,
        output_uri=output_uri,
        features=[videointelligence.enums.Feature.PERSON_DETECTION],
        video_context=context,
    )

    return operation


In [ ]:
operation = detect_person(input_uri, output_uri)

`detect_person` is an asynchronous function and it takes a minute or two for the entire video to be analyzed. You can check the status of the analysis by calling `operation.done`:

In [ ]:
print(f"Operation ${operation.operation.name} is done? {operation.done()}")

Note that even if you restart this notebook, the Video Intelligence API will still be analyzing your video in the cloud, so you won't lose any progress.

Rerun the previous command until it outputs `True`. Once the operation is finished, download the results from your Cloud Storage bucket:

In [ ]:
# Note! This won't work unless operation.done() == True!
!mkdir tmp
!gsutil cp {output_uri} tmp

## Formatting the Data

Results are written to Sloud Storage as a `json` file. Run this command to load them:

In [ ]:
data = json.load(open('./tmp/output.json'))

These `json` files are usually pretty big, so don't print them! Instead, inspect the structure:

In [ ]:
print(data.keys())
# You only care about annotation_results[0] because you only have one video
print(len(data['annotation_results'][0]['person_detection_annotations']))

It's easy to get lost in all of these nested fields, but the important data for this lab is stored in `data['annotation_results'][0]['person_detection_annotations']`. Store it in `people_annotations`:

In [ ]:
people_annotations = data['annotation_results'][0]['person_detection_annotations']

In `people_annotations`, every entry corresponds to a person, and each person has a unique set of `tracks`, or tracked segments. Write a helper function to parse through the data and rearrange it to make it easier to use:

In [ ]:
'''
This helper function takes in a person and rearranges the data so it's in 
a timeline, which will make it easier for us to work with
'''
def analyzePerson(person):
  frames = []
  for track in person['tracks']:
    # Convert timestamps to seconds
    for ts_obj in track['timestamped_objects']:
      time_offset = ts_obj['time_offset']
      timestamp = 0
      if 'nanos' in time_offset:
        timestamp += time_offset['nanos'] / 10**9
      if 'seconds' in time_offset:
        timestamp += time_offset['seconds']
      if 'minutes' in time_offset:
        timestamp += time_offset['minutes'] * 60
      frame= {'timestamp' : timestamp}
      for landmark in ts_obj['landmarks']:
        frame[landmark['name'] + '_x'] = landmark['point']['x']
        # Subtract y value from 1 because positions are calculated
        # from the top left corner
        frame[landmark['name'] + '_y'] = 1 - landmark['point']['y']
      frames.append(frame)
  sorted(frames, key=lambda x: x['timestamp'])
  return frames

Next, store the data in a pandas DataFrame (also for convenience), and sort each data point by timestamp:

In [ ]:
annotationsPd = pd.DataFrame(analyzePerson(people_annotations[0]))
for annotation in people_annotations[1:]:
  annotationsPd = annotationsPd.append(pd.DataFrame(analyzePerson(annotation)))

annotationsPd = annotationsPd.sort_values('timestamp', ascending=True)

Now, take a look at the results!

In [ ]:
annotationsPd.head()

As you can see above, the data is organized by the position of each body part by timestamp. Note: this works because there is only one person in the video used for this lab.

## Plotting the Data

With the video data formatted, plot the positions of the wrists and try to determine the start and end time of a serve:

In [ ]:
plt.figure()
annotationsPd.plot('timestamp', ['left_wrist_y', 'right_wrist_y'], figsize=(20, 5))
plt.title("Left and Right Wrist Positions Over Time")
plt.savefig("wrist_pos")

From the plot above, you can actually identify the time of a serve pretty easily! First, the tennis ball is thrown up with the left hand (peak in left wrist/blue line). Then, a few seconds later, the ball is hit with the racket (peak in right wrist/orange line).

The above plot is useful, but what would be even better would be understanding the angles of the elbow, knee, etc., in relation to the positions of the wrists.

## Computing Angles
Step 1: Create some classes for making working with points easier:

In [ ]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y 

To compute the angle made by three points, use the [Law of Cosines](https://www.google.com/search?q=rule+of+cosines&oq=rule+of+cosines&aqs=chrome.0.0l8.1048j1j7&sourceid=chrome&ie=UTF-8#wptab=s:H4sIAAAAAAAAAONgVuLQz9U3SMkuLnjE6Mgt8PLHPWEpi0lrTl5jNOLiCs7IL3fNK8ksqRRS4WKDsqS4eKTgmjQYpLi44DyeXUyCzvl5yakFJc6JOcmlOYkl-UWLWCV9EssV8tMUkvOLM_NSixWS4XIALyw914AAAAA). Imagine a triangle with side lengths a, b, and c. Then, to find 𝛾 (the angle across from side c), the formula is:

\begin{equation*}
\gamma = \cos^{-1}\frac{a^2+b^2 - c^2}{2ab}
\end{equation*}

The math is provided for you in this lab, but if it feels a bit confusing, there's a good explanation and code sample [here](https://medium.com/@manivannan_data/find-the-angle-between-three-points-from-2d-using-python-348c513e2cd), from which this function is borrowed:

In [ ]:
def getAngle(a, b, c):
    ang = math.degrees(math.atan2(c.y-b.y, c.x-b.x) - math.atan2(a.y-b.y, a.x-b.x))
    return ang

Using the `point` class and `getAngle` method, compute some useful angles below:

In [ ]:
def computeElbowAngle(row, which='right'):
  wrist = Point(row[f'{which}_wrist_x'], row[f'{which}_wrist_y'])
  elbow = Point(row[f'{which}_elbow_x'], row[f'{which}_elbow_y'])
  shoulder = Point(row[f'{which}_shoulder_x'], row[f'{which}_shoulder_y'])
  return getAngle(wrist, elbow, shoulder)

def computeShoulderAngle(row, which='right'):
  elbow = Point(row[f'{which}_elbow_x'], row[f'{which}_elbow_y'])
  shoulder = Point(row[f'{which}_shoulder_x'], row[f'{which}_shoulder_y'])
  hip = Point(row[f'{which}_hip_x'], row[f'{which}_hip_y'])
  return getAngle(hip, shoulder, elbow)

def computeKneeAngle(row, which='right'):
  hip = Point(row[f'{which}_hip_x'], row[f'{which}_hip_y'])
  knee = Point(row[f'{which}_knee_x'], row[f'{which}_knee_y'])
  ankle = Point(row[f'{which}_ankle_x'], row[f'{which}_ankle_y'])
  return getAngle(ankle, knee, hip)

In [ ]:
# For a single timeslot...
row = annotationsPd.iloc[-1]
print("Elbow angle: " + str(computeElbowAngle(row)))
print("Shoulder angle: " + str(computeShoulderAngle(row)))
print("Knee angle: " + str(computeKneeAngle(row)))

Now, plot those angles over time.

In [ ]:
annotationsPd['right_elbow_angle'] = annotationsPd.apply(computeElbowAngle, axis=1)
annotationsPd['right_shoulder_angle'] = annotationsPd.apply(computeShoulderAngle, axis=1)
annotationsPd['right_knee_angle'] = annotationsPd.apply(computeKneeAngle, axis=1)

Next, plot the results!

In [ ]:
plt.figure()
annotationsPd.plot('timestamp', ['right_elbow_angle'], figsize=(20, 5), color='blue')
plt.title("Right Elbow Angle over Time")
plt.savefig("right_elbow_angle")
annotationsPd.plot('timestamp', ['right_shoulder_angle'], figsize=(20, 5), color='purple')
plt.title("Right Shoulder Angle over Time")
plt.savefig("right_shoulder_angle")
annotationsPd.plot('timestamp', ['right_knee_angle'], figsize=(20, 5))
plt.title("Right Knee Angle over Time")
plt.savefig("right_knee_angle")

These angles might not be very useful on their own, but when combined with position data, you can tell what the angle of the arm was _at the height of the serve_. In particular, take a look at the angle of the elbow and shoulder when the right wrist was at the highest point in the serve:

In [ ]:
fig = plt.figure()
ax=fig.add_subplot(111, label="1")
annotationsPd.plot('timestamp', ['right_wrist_y'], figsize=(20, 5), ax=ax, color='red')
plt.title("Right Elbow Angle over Time")

ax2=fig.add_subplot(111, label="2", frame_on=False)
annotationsPd.plot('timestamp', ['right_elbow_angle'], figsize=(20, 5), ax=ax2)

#annotationsPd.plot.scatter('right_wrist_y', 'right_elbow_angle')

These charts might be difficult to read, but but they show that the elbow is at a 200 degree angle when the arm is most extended. In an ideal tennis serve, the elbow would be straight when the arm is most extended.

# Tracking the Speed of the Ball With AutoML

To compute the speed of the ball, you will use [AutoML Vision Object Detection](https://cloud.google.com/vision/automl/object-detection/docs). Training an AutoML Vision model can be a long process, so for this lab, you will send an authenticated Cloud Run request to a pretrained model.

Check out the [blog post](https://daleonai.com/machine-learning-for-sports) if you are curious about how the model was trained.

To make a prediction with AutoML, first convert the video of tennis serves into images. Download the video from the public bucket:

In [ ]:
!gsutil cp gs://spls/aiforsports/tennis_serves.mp4 .
filename = "tennis_serves.mp4"

Use the `ffmpeg` command to generate snapshots from the tennis serve video at 20 frames per second. The `-t` flag of this command specifies a 2 second segment (`-t 00:00:02`) that starts from one second in (`-ss 00:00:01`). This should align with the first serve.

In [ ]:
!mkdir tmp/snapshots
!ffmpeg -i {filename} -vf fps=20 -ss 00:00:01 -t 00:00:02 tmp/snapshots/%03d.jpg

For ease of use, store the snapshots in a variable and sort them:

In [ ]:
snapshotFiles = os.listdir('tmp/snapshots')
snapshotFiles.sort()
print(f"Analyzing {len(snapshotFiles)} images")

Now, upload these snapshots back into your bucket so that they will be easy to use in the Cloud Run prediction request:

In [ ]:
snapshot_bucket_location = f"{bucket}/snapshots"
!gsutil -m cp ./tmp/snapshots/* {snapshot_bucket_location}

Next, write the prediction function which uses the AutoML model to analyze those snapshots. In order to do that, you will need a valid ID token to access the pretrained model.

Access your student ID token by logging in to [this authentication page](https://gsp-auth-kjyo252taq-uc.a.run.app/) with your student email address and password.

Once logged in click the __Copy__ button, paste it into this id_token variable and run the cell:

In [ ]:
id_token = "PASTE_YOUR_ID_TOKEN_HERE"

__If you don't see an ID token after logging in, you probably need to allow cookies for that page in your browser. This will consistently happen if running in incognito mode. In Chrome, cookies can be enabled by clicking the eye icon to the far right of your URL tab.__

Now, you are able to analyze the snapshots. Write the `getAutoMLPrediction` function:

In [ ]:
from google.cloud import storage
import re
import logging
import requests

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def getAutoMLPrediction():
    responseList = []
    match = re.match(r'gs://([^/]+)/(.+)', snapshot_bucket_location)
    bucket_name = match.group(1)
    prefix = match.group(2)
    storage_client = storage.Client.from_service_account_json("./key.json") 
    my_bucket = storage_client.bucket(bucket_name)
    for blob in my_bucket.list_blobs(prefix=str(prefix + "/")):
        if blob.name.endswith(".jpg"):
            logger.info("File location: {}".format(os.path.join('gs://',bucket_name, blob.name)))
            params = {}
            
            #[START] GSP666-API REQUEST
            url_lifetime = 3600  # Seconds in an hour
            serving_url = blob.generate_signed_url(expiration=url_lifetime, version='v4')
            # response = prediction_client.predict(model_full_id, payload, params)
            header = {'Authorization': 'Bearer ' + id_token}
            data = {"image_url" : serving_url,
                "compute_region": "us-central1",
                "model_id": "IOD5928674449107714048",
                }
            url = 'https://gsp666-api-kjyo252taq-uc.a.run.app/image'
            response = requests.post(url, json=data, headers=header)
            response_data = response.json()
            logger.info("response data from API")
            logger.info(response_data)
            #[END] GSP666-API REQUEST
            if "payload" in  response_data:
                responseList.append([obj['imageObjectDetection']['boundingBox']['normalizedVertices'] for obj in response_data['payload']])
            else:
                responseList.append({})
            
    return responseList

The `getAutoMLPredicion` function loops through your snapshots and generates a payload (the data variable) from the snapshot's url within the bucket, the compute region (AutoML predict only supports us-central1 at the moment), and the model ID of the pretrained model. The function uses the ID token you stored earlier to authorize this Cloud Run request to the model.

The response from the Cloud Run request is the response from the pretrained model that is exposed via Cloud Run. When you run the function, the logs will display the information in each response object.

With the __getAutoMLPrediciton__ function written, run it and assign its return value (the responses) to the coords variable: 

In [ ]:
# Call the AutoML API--this could take a while!
coords = getAutoMLPrediction()

Don't be alarmed by snapshots that return empty responses. If the prediction score of the model was too low, it will return an empty response. In this case, this is usually because there is no tennis ball in some of the snapshots but can also be the result of the small margin of error the model has.

Now with the AutoML data, create an image to see what's actually going on:

In [ ]:
def makeBallImage(filename, coords):
  im = Image.open(filename)
  im.thumbnail((im.width * 0.2, im.height * 0.2))
  draw = ImageDraw.Draw(im)
  for coord in coords:
    draw.rectangle([(coord[0]['x'] * im.width, coord[0]['y'] * im.height), coord[1]['x'] * im.width, coord[1]['y'] * im.height])
  return im

In [ ]:
imgs = [makeBallImage('tmp/snapshots/' + filename, coord) for filename, coord in zip(snapshotFiles, coords) if 'jpg' in filename]
!mkdir snapshot_annotated
for idx, im in enumerate(imgs):
  plt.imshow(np.asarray(im))
  plt.savefig('snapshot_annotated/file%d.png' % idx)
  
# Create a cute video of your serves!
!ffmpeg -framerate 20 -i snapshot_annotated/file%01d.png -vcodec mpeg4 -y ball_tracking.mp4
!ffmpeg -i ball_tracking.mp4 ball_tracking.gif

The code above analyzes the snapshots and creates a gif and video you can check out in the files `ball_tracking.mp4` and `ball_tracking.gif` respectively.

Now that the ball is tracked, you can compute it's position and then speed!

In [ ]:
# For simplicity, just plot the bottom left corner of the bounding box
# around the ball
coord_x = [ball[0]['x'] for frame in coords for ball in frame]
coord_y = [1 - ball[0]['y'] for frame in coords for ball in frame]
timestamps = [x/20 for x in range(len(coord_x))] # 20 frames per second

Plot the ball in space, and see how it leaves the hand and then flies across the court.

In [ ]:
plt.title("Position of tennis ball during serve")
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.scatter(coord_x, coord_y)
plt.savefig("serve_position_x_y.png")

To determine the speed, look at the distance the ball travels over time:

In [ ]:
plt.title("Y position of tennis ball during serve over time")
plt.xlabel("seconds")
plt.ylabel("Y position")
plt.scatter(timestamps, coord_y)
plt.savefig("ball_position_over_time.png")

You can see that 0.5 to 0.7 seconds is when the ball has been hit and is traveling across the court. So, to compute the speed, divide distance by time!

In [ ]:
# Get the first data point from 0.5 seconds
start_x = coord_x[timestamps.index(0.5)]
end_x = coord_x[-1]
start_y = coord_y[timestamps.index(0.5)]
end_y = coord_y[-1]

# Compute the Euclidean distance
distance = math.sqrt((start_x - end_x)**2 + (start_y - end_y)**2)
time = timestamps[-1] - 0.5

print(f"The speed of your serve was {distance/time}")